## This code is an exact implementation of a research paper on Astute RAG (Retrieval-Augmented Generation). 

## https://arxiv.org/pdf/2410.07176

## ASTUTE RAG

The Astute RAG method is designed to address issues in Retrieval-Augmented Generation (RAG) where retrieval from external sources may provide unreliable or misleading information. This leads to knowledge conflicts with an LLM’s internal knowledge, reducing answer reliability.
Astute RAG takes three key steps to improve RAG’s performance by integrating internal and external information selectively and iteratively to filter out low-quality data:
1. Adaptive Internal knowledge generation: The LLM generates passages from its internal knowledge based on the query. The LLM will have the ability to decide the number of passages to generate( with a predefined upper limit). These passages are appended into a list along with the passages retrieved from external sources.


2. Iterative Source-aware Knowledge Consolidation: For each passage in the list, we additionally provide the source of generation for the LLM to check its reliability. The objective of this step is to iteratively improve the response quality over each iteration and produce fewer and refined passages. 

In each iteration, the LLM consolidates information by:
Grouping Consistent Information: The LLM clusters passages that share similar details or answer elements, producing a consolidated, refined passage representing the consistent information.
Identifying Conflicts: When passages disagree (e.g., they present different facts or perspectives), they are separated and labelled as conflicting. This way, the model can weigh each conflicting source’s reliability, rather than combining contradictory information.
Filtering Irrelevant Information: Passages irrelevant to the query are excluded to avoid distractions or inaccuracies.


The above steps repeat for a set number of iterations (with an upper bound parameter t ). Each cycle progressively refines the knowledge pool by further consolidating consistent data, reducing irrelevant information, and highlighting conflicts until the information pool is accurate, consistent, and minimal in contradictions.


   3. Answer Finalisation: The refined passages from the above step are now used for the final answer generation.  For each group of passages (from the consolidated knowledge), the model generates potential answers, ensuring that different perspectives are considered separately. Each answer is assigned a confidence score based on factors like reliability of source and persistence across sources. The LLM evaluates all the proposed answers and selects the one with the highest confidence score as the final answer.

## Code Overview
This Python script implements a Retrieval-Augmented Generation (RAG) model for knowledge consolidation using external and internal document sources. It integrates OpenAI's GPT-3.5 model with LangChain to perform document search, generate internal knowledge passages, merge documents, and provide consolidated answers to user queries.

## Key Features:
Document Retrieval: It uses TavilySearchResults to search external documents based on the query and retrieve relevant content.

Internal Document Generation: It generates internal knowledge passages using OpenAI's GPT-3.5 model, based on an initial context.

Document Merging: The script merges internal and external documents into a consolidated list.

Knowledge Consolidation: It iteratively consolidates information, ensuring consistency and handling conflicting data.

Answer Generation: Finally, the consolidated knowledge is used to generate an answer to the user's query.

## Process Flow:
Document Search: Retrieve external information related to the user’s query.

Internal Passage Generation: Generate additional internal knowledge passages using GPT-3.5.

Document Merging: Combine both internal and external documents into a single list.

Consolidation: Use iterative calls to GPT-3.5 to consolidate information, resolving inconsistencies(Implementation of Astute Rag).

Answer Generation: Generate a final, consolidated answer based on the merged and refined knowledge.

## Requirements:
langchain_openai for OpenAI's integration with LangChain.

langchain_community for TavilySearch and other community tools.

OpenAI API Key for accessing GPT-3.5 model.

In [5]:
# !pip install langchain langchain-openai langchain-community openai tavily-python

In [2]:
import os
import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass()
model = ChatOpenAI(model="gpt-3.5-turbo")

 ········


In [3]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] ="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"

In [6]:
# Import necessary libraries
import openai
from langchain import OpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Set your OpenAI API key
openai.api_key = 'YOUR_API_KEY'  # Replace with your actual API key

# Step 1: Define the search tool
search = TavilySearchResults(max_results=2)

# Function to perform the search and retrieve documents
def retrieve_documents(query):
    # Assuming search_results can have dictionaries or strings
    search_results = search.invoke(query)  # Adjust this to however you're obtaining search results
    external_docs = []
    for result in search_results:
        if isinstance(result, dict):
            content = result.get("content", "No content available")
            source = result.get("source", "External")  # Default source if not available
        else:  # If result is a string, assume it's content with no source information
            content = result
            source = "External"
        external_docs.append({"content": content, "source": source})
    return external_docs

# Step 2: Internal knowledge generation using OpenAI LLM
# Function to generate internal passages

from langchain_openai import OpenAI

client = OpenAI(model="gpt-3.5-turbo")

# Updated code for using chat completions
from langchain_community.llms import OpenAI
import os
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def generate_internal_passages(initial_context):
    prompt = f"""
    Generate additional internal passages related to the provided context. 
    Use the following initial context for inspiration:
    '{initial_context}'
    Please provide concise passages that cover various relevant aspects based on the context given.
    """
    
    # Using the new method to create a chat completion
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150
    )
    
    # Extract the text from the response
    internal_passages = response.choices[0].message.content.strip().split('\n')
    return internal_passages



# Step 3: Merging internal and external documents
def merge_documents(internal_docs, external_docs):
    merged_docs = []
    for doc in internal_docs:
        if isinstance(doc, dict):  # Check if doc is a dictionary
            merged_docs.append({"content": doc["content"], "source": doc.get("source", "internal")})
        else:
            merged_docs.append({"content": doc, "source": "internal"})  # Handle string case

    for doc in external_docs:
        merged_docs.append({"content": doc["content"], "source": doc["source"]})

    return merged_docs

# Step 4: Define the prompt for knowledge consolidation
def create_consolidation_prompt(merged_docs, query):
    merged_docs_str = "\n".join([f"{doc['content']} [Source: {doc['source']}]" for doc in merged_docs])
    return f"""
    Task: Consolidate information from the provided documents in response to the given question.

    * For documents that provide consistent information, cluster them together and summarize the key details into a single, concise document.
    * For documents with conflicting information, separate them into distinct documents, ensuring each captures the unique perspective or data.
    * Exclude any information irrelevant to the query.

    For each new document created, clearly indicate:
    * Whether the source was from memory or an external retrieval.
    * The original document numbers for transparency.

    Merged Documents: {merged_docs_str}
    Question: {query}
    New Context:
    """

# Step 5: Call OpenAI's LLM to generate answers
def generate_answer(consolidated_docs, query, initial_context):
    context = "\n".join([f"{doc['source']}: {doc['content']}" for doc in consolidated_docs])
    prompt = f"""
    Task: Answer the following question using the consolidated information from internal and external documents.

    Initial Context: {initial_context}
    Consolidated Context: {context}
    Question: {query}
    Answer:
    """
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content.strip().split('\n')

# Main function to execute the entire process
def main():
    # Ask the user for their query
    query = input("Please enter your query: ")
    initial_context = "You are a helpful agent"  # You can modify this as needed

    # Step 1: Retrieve external documents
    external_docs = retrieve_documents(query)
    
    # Step 2: Generate internal documents using OpenAI
    internal_docs = generate_internal_passages(query)
    
    # Step 3: Merge internal and external documents
    merged_docs = merge_documents(internal_docs, external_docs)  # Initialize merged_docs here
    print("Initial merged_docs:", merged_docs)  # Debug: Verify merged_docs initialization

    # Step 4 & 5: Iteratively consolidate information
    num_iterations = 10
    
    # Create an instance of OpenAI client
    llm = client  # This should refer to the correct OpenAI instance, not `model`
    
    for i in range(num_iterations):
        merged_docs_str = "\n".join([f"{doc['content']} [Source: {doc['source']}]" for doc in merged_docs])
        
        # Use the prompt consolidation here
        consolidation_prompt = f"""
        Task: Consolidate information from the provided documents in response to the given question.

        * For documents that provide consistent information, cluster them together and summarize the key details into a single, concise document.
        * For documents with conflicting information, separate them into distinct documents, ensuring each captures the unique perspective or data.
        * Exclude any information irrelevant to the query.

        Merged Documents: {merged_docs_str}
        Question: {query}
        New Context:
        """
        
        # Call the LLM to consolidate information
        response = llm.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[{"role": "user", "content": consolidation_prompt}],
            max_tokens=150  # Adjust max_tokens as needed
        )
        
        consolidated_info = response.choices[0].message.content.strip()
        print(f"Iteration {i + 1} Consolidated Info:\n{consolidated_info}\n")
        
        # Update the merged_docs for the next iteration
        merged_docs = [{"content": line.strip(), "source": "Consolidated"} for line in consolidated_info.split('\n') if line.strip()]

    # Final answer generation
    final_answer = generate_answer(merged_docs, query, initial_context)
    
    # Print the final answer
    print(f"Final Answer:\n{final_answer}")

# Example usage
if __name__ == "__main__":
    main()


Please enter your query:  hi


Initial merged_docs: [{'content': '- Hello, how are you today?', 'source': 'internal'}, {'content': "- Hi there, what's on your mind?", 'source': 'internal'}, {'content': "- Hey, haven't heard from you in a while.", 'source': 'internal'}, {'content': '- Hi, do you have a moment to chat?', 'source': 'internal'}, {'content': '- Hi friend, just checking in.', 'source': 'internal'}, {'content': 'H', 'source': 'External'}, {'content': 'T', 'source': 'External'}, {'content': 'T', 'source': 'External'}, {'content': 'P', 'source': 'External'}, {'content': 'E', 'source': 'External'}, {'content': 'r', 'source': 'External'}, {'content': 'r', 'source': 'External'}, {'content': 'o', 'source': 'External'}, {'content': 'r', 'source': 'External'}, {'content': '(', 'source': 'External'}, {'content': "'", 'source': 'External'}, {'content': '4', 'source': 'External'}, {'content': '0', 'source': 'External'}, {'content': '0', 'source': 'External'}, {'content': ' ', 'source': 'External'}, {'content': 'C', '